In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

in_session_median_embeddings_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/in_session_median_embeddings'
os.makedirs(in_session_median_embeddings_data_dir, exist_ok=True)

in_session_cluster_embeddings_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/in_session_cluster_embeddings'
os.makedirs(in_session_cluster_embeddings_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_tracking/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)


In [ ]:
sessionA = 'classinsight-cmu_05748A_ghc_4101_201902141630'
sessionB = 'classinsight-cmu_05748A_ghc_4101_201902281630'
course = '05748A'


In [ ]:
sessionA_frame_dir = f'{base_dir}/{course}/{sessionA}-front'
sessionB_frame_dir = f'{base_dir}/{course}/{sessionB}-front'


## Get frame file data for all sessions

In [ ]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()


## Get embedding and gaze information for all frames for all sessions (Run if needed, commented out for now)?

In [ ]:
sessionA_emb_info = pickle.load(open(f'{emb_analysis_session_data}/{sessionA}-front.pb','rb'))
sessionB_emb_info = pickle.load(open(f'{emb_analysis_session_data}/{sessionB}-front.pb','rb'))

sessionA_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{sessionA}-front.pb","rb"))
sessionB_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{sessionB}-front.pb","rb"))

df_sessionA_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{sessionA}-front.csv")
df_sessionB_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{sessionB}-front.csv")

sessionA_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sessionA}/{sessionA}-front.avi'
sessionB_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sessionB}/{sessionB}-front.avi'


## Replace raw ids with mapped ids after postprocessing for both sessions

In [ ]:
sessionA_emb_info = {
    xr:{
        sessionA_id_map[yr]:sessionA_emb_info[xr][yr] 
            for yr in sessionA_emb_info[xr] if not (sessionA_id_map[yr]==10000)} for xr in sessionA_emb_info}


In [ ]:
sessionB_emb_info = {
    xr:{
        sessionB_id_map[yr]:sessionB_emb_info[xr][yr] 
            for yr in sessionB_emb_info[xr] if not (sessionB_id_map[yr]==10000)} for xr in sessionB_emb_info}


In [ ]:
len(sessionA_emb_info.keys()), len(sessionB_emb_info.keys())


# Correct new ids with dict from eligible pairs

In [ ]:
eligible_id_map_dictA = {}
for id_pair in df_sessionA_eligible_pairs.id_pair.values:
    (id1, id2) = eval(id_pair)
    print(id1, id2)
    if id2 in eligible_id_map_dictA:
        eligible_id_map_dictA[id1] = eligible_id_map_dictA[id2]
    else:
        eligible_id_map_dictA[id2] = id1

eligible_id_map_dictB = {}
for id_pair in df_sessionB_eligible_pairs.id_pair.values:
    (id1, id2) = eval(id_pair)
    if id2 in eligible_id_map_dictB:
        eligible_id_map_dictB[id1] = eligible_id_map_dictB[id2]
    else:
        eligible_id_map_dictB[id2] = id1
sprint(eligible_id_map_dictA)
sprint(eligible_id_map_dictB)


In [ ]:
df_sessionA_eligible_pairs


In [ ]:
df_sessionB_eligible_pairs


In [ ]:
eval(df_sessionA_eligible_pairs.id_pair.values[0])


In [ ]:
# arrange info as per tracking id across both sessions
# pitch, roll, yaw= sessionA_emb_info[0][9]['rvec'][0]
gaze_infoA = {}
emb_infoA = {}
bbox_infoA = {}
for frame_number in sessionA_emb_info:
    for trackId_old in sessionA_emb_info[frame_number]:
        trackId = trackId_old if (trackId_old not in eligible_id_map_dictA.keys()) else eligible_id_map_dictA[trackId_old]
        if trackId not in gaze_infoA:
            gaze_infoA[trackId] = []
            emb_infoA[trackId]=[]
            bbox_infoA[trackId] = []
        # get  gaze info
        try:
            id_bbox = sessionA_emb_info[frame_number][trackId]['bbox']
            bbox_infoA[trackId].append([frame_number]+list(id_bbox))
            pitch, roll, yaw= sessionA_emb_info[frame_number][trackId]['rvec'][0]
            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
            gaze_sx, gaze_sy, gaze_ex, gaze_ey = sessionA_emb_info[frame_number][trackId]['gaze_2d'][0].flatten()
            gaze_infoA[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
            face_emb = sessionA_emb_info[frame_number][trackId]['face_embedding'].tolist()
            emb_infoA[trackId].append([frame_number]+face_emb)
        except:
            continue

for id in gaze_infoA:
    gaze_infoA[id] = pd.DataFrame(gaze_infoA[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
    emb_infoA[id] =pd.DataFrame(emb_infoA[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
    bbox_infoA[id] = pd.DataFrame(bbox_infoA[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')

gaze_infoB = {}
emb_infoB = {}
bbox_infoB = {}
for frame_number in sessionB_emb_info:
    for trackId_old in sessionB_emb_info[frame_number]:
        trackId = trackId_old if (trackId_old not in eligible_id_map_dictB.keys()) else eligible_id_map_dictB[trackId_old]
        if trackId not in gaze_infoB:
            gaze_infoB[trackId] = []
            emb_infoB[trackId]=[]
            bbox_infoB[trackId] = []
        # get  gaze info
        try:
            id_bbox = sessionB_emb_info[frame_number][trackId]['bbox']
            bbox_infoB[trackId].append([frame_number]+list(id_bbox))
            pitch, roll, yaw= sessionB_emb_info[frame_number][trackId]['rvec'][0]    
            gaze_sx, gaze_sy, gaze_ex, gaze_ey = sessionB_emb_info[frame_number][trackId]['gaze_2d'][0].flatten()
            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
            gaze_infoB[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
            face_emb = sessionB_emb_info[frame_number][trackId]['face_embedding'].tolist()
            emb_infoB[trackId].append([frame_number]+face_emb)
        except:
            continue


for id in gaze_infoB:
    gaze_infoB[id] = pd.DataFrame(gaze_infoB[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
    emb_infoB[id] =pd.DataFrame(emb_infoB[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
    bbox_infoB[id] = pd.DataFrame(bbox_infoB[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')

len(gaze_infoA.keys()), len(gaze_infoB.keys())


In [ ]:
sprint({xr:(gaze_infoA[xr].shape[0],emb_infoA[xr].shape[0], bbox_infoA[xr].shape[0]) for xr in emb_infoA})


In [ ]:
sprint({xr:(gaze_infoB[xr].shape[0],emb_infoB[xr].shape[0], bbox_infoB[xr].shape[0]) for xr in emb_infoB})


# Ground truth for id matching

In [ ]:
sprint(eligible_id_map_dictA)
sprint(eligible_id_map_dictB)


In [ ]:
gt_str_new = '''
3,12
11,17
5,10
5,2
9,9
1,0
6,8
4,8
14,1
16,6
21,4
7,2
0,5
-1,20
13,3
10,7
19,14
19,11
15,-1
2,-1
8,-1
-1,16
-1,18
'''


In [ ]:
gt_arr = gt_str.split("\n")[1:-1]
gt_arr = [xr.split(",") for xr in gt_arr]
df_gt = pd.DataFrame(gt_arr, columns=[sessionA,sessionB])
gt_map = df_gt.groupby(sessionA).agg({sessionB:lambda x: list(x)})[sessionB].to_dict()
gt_map


# Clean Implementation 1: Using k% gaze filtering and median embeddings for matching

In [ ]:
MAX_GAZE_DEVIATION_DEG = 30
# MAX_EMBEDDING_FRAMES = 1000
MIN_EMBEDDING_FRAMES = 100


In [ ]:
match_scores = {}
match_info = {}
np.random.seed(42)
for idA,idB in product(emb_infoA.keys(), emb_infoB.keys()):
    #filter correct frames
    framesA = gaze_infoA[idA][
        (gaze_infoA[idA].yaw.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].pitch.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values
    framesB = gaze_infoB[idB][
        (gaze_infoB[idB].yaw.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].pitch.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values

    num_framesA, num_framesB = len(framesA), len(framesB)
    if (num_framesA<MIN_EMBEDDING_FRAMES) | (num_framesB <MIN_EMBEDDING_FRAMES):
        sprint(f"Not sufficient frames to match {idA}:{len(framesA)},{idB}:{len(framesB)}")
        continue
    
    # select_framesA  =  np.random.choice(framesA, MAX_EMBEDDING_FRAMES) if (num_framesA > MAX_EMBEDDING_FRAMES) else  framesA
    # select_framesB  =  np.random.choice(framesB, MAX_EMBEDDING_FRAMES) if (num_framesB > MAX_EMBEDDING_FRAMES) else  framesB

    #get id embeddings
    
    medianA_emb = np.median(emb_infoA[idA].loc[framesA],axis=0)
    medianB_emb = np.median(emb_infoB[idB].loc[framesB],axis=0)
    match_distance = cdist(medianA_emb.reshape(1,-1), medianB_emb.reshape(1,-1))[0][0]
    if idA not in match_scores:
        match_scores[idA] = {}
        match_info[idA] = {}
    match_scores[idA][idB] = match_distance
    match_info[idA][idB] = dict(match_score=match_distance, framesA=framesA, framesB=framesB)
    sprint(idA, idB, match_distance)
    
    
    
    

In [ ]:
df_matching_methodA = pd.DataFrame(match_scores)
df_matching_methodA.shape


In [ ]:
df_matching_methodA


In [ ]:
from matplotlib.patches import Rectangle
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching_methodA.round(2), annot=True,ax=axn,cmap='bone_r')
# for gtA in map(int, gt_map):
#     for gtB in map(int, gt_map[str(gtA)]):
#         if (gtA>=0) and (gtB>=0):
#             if (gtA in df_matching_methodA.columns) and (gtB in df_matching_methodA.index):
#                 locA, locB = df_matching_methodA.columns.get_loc(gtA), df_matching_methodA.index.get_loc(gtB)
#                 axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching_methodA.shape[1]):
    locBs = df_matching_methodA.iloc[:,locA].argsort()[:2]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))

# 5, 11, 12, 12 
# absent: 5 out of 20 (how??)
# 
axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method1_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


# Method 2:Get distance between all pairs and get median distance across all pairs. 

In [ ]:
MAX_GAZE_DEVIATION_DEG = 30 # for basic filtering of very random gaze
MAX_EMBEDDING_FRAMES = 1000
MIN_EMBEDDING_FRAMES = 100


In [ ]:
match_scores = {}
for idA in matching_info_dictB:
    match_scores[idA] = {}
    for idB in matching_info_dictB[idA]:
        match_matrix = matching_info_dictB[idA][idB]['match_matrix']
        match_distance = np.median(np.median(match_matrix,axis=1))
        match_scores[idA][idB] = match_distance

df_matching = pd.DataFrame(match_scores)

#--------
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching.columns) and (gtB in df_matching.index):
                locA, locB = df_matching.columns.get_loc(gtA), df_matching.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching.shape[1]):
    locBs = df_matching.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method2b_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


# Method 3: Use spectral clustering to get clean set of embeddings, calculate their centroid, and then evaluate distance

In [ ]:
from sklearn.cluster import DBSCAN
EPS = 0.4
MIN_PTS = 100


In [ ]:

np.random.seed(42)
clustered_median_embA = {}
for idA in emb_infoA:
    emb_cluA = DBSCAN(min_samples=MIN_PTS, eps=EPS)
    emb_cluA.fit(emb_infoA[idA].values)
    if max(emb_cluA.labels_)<0:
        sprint(f"All frames are outliers, not proceeding with id {idA}")
        continue
    best_cluster_id = pd.Series(emb_cluA.labels_[emb_cluA.labels_>=0]).value_counts().index[0]
    framesA = emb_infoA[idA].iloc[emb_cluA.labels_==best_cluster_id].index.values
    clustered_median_embA[idA] = np.median(emb_infoA[idA].loc[framesA],axis=0)
    
clustered_median_embB = {}
for idB in emb_infoB:
    emb_cluB = DBSCAN(min_samples=MIN_PTS, eps=EPS)
    emb_cluB.fit(emb_infoB[idB].values)
    if max(emb_cluB.labels_)<0:
        sprint(f"All frames are outliers, not proceeding with id {idB}")
        continue
    best_cluster_id = pd.Series(emb_cluB.labels_[emb_cluB.labels_>=0]).value_counts().index[0]
    framesB = emb_infoB[idB].iloc[emb_cluB.labels_==best_cluster_id].index.values
    clustered_median_embB[idB] = np.median(emb_infoB[idB].loc[framesB],axis=0)
    

In [ ]:
match_scores = {}
match_info3 = {}
for idA in clustered_median_embA:
    match_scores[idA] = {}
    for idB in clustered_median_embB:
        match_scores[idA][idB] = cdist(clustered_median_embA[idA].reshape(1,-1), clustered_median_embB[idB].reshape(1,-1))[0][0]
        
df_matching_methodC = pd.DataFrame(match_scores)
df_matching_methodC.shape    


In [ ]:
from matplotlib.patches import Rectangle
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching_methodC.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching_methodC.columns) and (gtB in df_matching_methodC.index):
                locA, locB = df_matching_methodC.columns.get_loc(gtA), df_matching_methodC.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching_methodC.shape[1]):
    locBs = df_matching_methodC.iloc[:,locA].argsort()[:4]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))

# 4, 8, 10, 11 ...  
# absent: 5 out of 20 (how??)
# 

axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method3_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')
